In [1]:
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd

from openoa.types.plant_v2 import PlantDataV3, PlantMetaData

In [2]:
fpath = Path("/Users/rhammond/Documents/GitHub/OpenOA/examples/data/la_haute_borne")
fn_scada = "la-haute-borne-data-2014-2015.csv"
fn_meter = "plant_data.csv"
fn_curtail = "plant_data.csv"
fn_reanalysis = "merra2_la_haute_borne.csv"
fn_asset = "la-haute-borne_asset_table.csv"

scada = pd.read_csv(fpath / fn_scada)
meter = pd.read_csv(fpath / fn_meter)
curtail = pd.read_csv(fpath / fn_curtail)
reanalysis = pd.read_csv(fpath / fn_reanalysis)
asset = pd.read_csv(fpath / fn_asset)

In [3]:
column_mapping = dict(
    scada=dict(
        time="Date_time",
        id="Wind_turbine_name",
        power="P_avg",
        windspeed="Ws_avg",
        wind_direction="Wa_avg",
#         status="?",
        pitch="Ba_avg",
        temperature="Ot_avg",
        frequency="10T",
    ),
    meter=dict(
        time="time_utc",
#         power="?",
        energy="net_energy_kwh",
    ),
    curtail=dict(
        time="time_utc",
        curtailment="curtailment_kwh",
        availability="availability_kwh",
        net_energy="net_energy_kwh",
        frequency="10T",
    ),
    reanalysis=dict(
        time="datetime",
        windspeed="windspeed_ms",
        windspeed_u="u_ms",
        windspeed_v="v_ms",
        temperature="temperature_K",
        rho="rho_kgm",
        frequency="10T",
#         product="merra2",
    ),
    asset=dict(
        id="id",
        latitude="latitude",
        longitude="longitude",
        rated_power="rated_power_kw",
#         type="?",
    ),
)

In [4]:
plant_meta = PlantMetaData.from_dict(column_mapping)

In [5]:
plant = PlantDataV3(
    metadata=column_mapping,
    scada=scada,
    meter=meter,
    curtail=curtail,
    reanalysis=reanalysis,
    asset=asset,
    analysis_type="MonteCarloAEP"
)

In [7]:
plant.analysis_type = "all"
plant.validate()

ValueError: `scada` data is missing the following columns: ['status']
`meter` data is missing the following columns: ['power']
`tower` data is missing the following columns: ['time', 'id']
`asset` data is missing the following columns: ['id', 'longitude', 'type', 'rated_power_kw', 'latitude']
`status` data is missing the following columns: ['time', 'id', 'status_id', 'status_code', 'status_text']
`reanalysis` data is missing the following columns: ['windspeed_ms', 'wind_direction', 'rho_kgm']
`scada` data columns were of the wrong type: ['status']
`meter` data columns were of the wrong type: ['power']
`tower` data columns were of the wrong type: ['time', 'id']
`asset` data columns were of the wrong type: ['id', 'latitude', 'longitude', 'rated_power_kw', 'type']
`status` data columns were of the wrong type: ['time', 'id', 'status_id', 'status_code', 'status_text']
`reanalysis` data columns were of the wrong type: ['windspeed_ms', 'wind_direction', 'rho_kgm']

In [8]:
plant.analysis_type = "TurbineLongTermGrossEnergy"
plant.validate()

ValueError: `reanalysis` data is missing the following columns: {'wind_direction'}
`reanalysis` data columns were of the wrong type: {'wind_direction'}

In [9]:
plant.analysis_type = "ElectricalLosses"
plant.validate()

In [ ]:
ANALYSIS_REQUIREMENTS = {
    "MonteCarloAEP": {
        "meter": {
            "columns": ["energy"],
            "freq": ("MS", "D", "H", "T"),
        },
        "curtail": {
            "columns": ["availability", "curtailment"],
            "freq": ("MS", "D", "H", "T"),
        },
        "reanalysis": {
            "columns": ["windspeed", "rho"],
            "conditional_columns": {
                "reg_temperature": ["temperature"],
                "reg_winddirection": ["windspeed_u", "windspeed_v"],
            },
        },
    },
    "TurbineLongTermGrossEnergy": {
        "scada": {
            "columns": ["id", "windspeed", "power"],  # TODO: wtur_W_avg vs energy_kwh ?
            "freq": ("D", "H", "T"),
        },
        "reanalysis": {
            "columns": ["windspeed", "wind_direction", "rho"],
        },
    },
    "ElectricalLosses": {
        "scada": {
            "columns": ["energy"],
            "freq": ("D", "H", "T"),
        },
        "meter": {
            "columns": ["energy"],
            "freq": ("MS", "D", "H", "T"),
        },
    },
}